**Install libraries**

In [11]:
pip install transformers datasets torch peft accelerate


**Load dataset from Hugging Face**

In [13]:
from datasets import load_dataset

dataset = load_dataset("blended_skill_talk")
print(dataset)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/980 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 4819
    })
    validation: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
        num_rows: 980
    })
})


**Convert dialog → chatbot text**

In [17]:
print(dataset["train"].column_names)


['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates']


In [18]:
def dialog_to_text(example):
    conversation = ""

    # context is a list of utterances
    for i, utterance in enumerate(example["context"]):
        if i % 2 == 0:
            conversation += f"User: {utterance}\n"
        else:
            conversation += f"Bot: {utterance}\n"

    # add bot response (free message)
    if len(example["free_messages"]) > 0:
        conversation += f"Bot: {example['free_messages'][0]}\n"

    return {"text": conversation}


In [19]:
dataset = dataset.map(dialog_to_text)


Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

In [20]:
print(dataset["train"][0]["text"])


User: e
Bot: m
User: p
Bot: a
User: t
Bot: h
User: e
Bot: t
User: i
Bot: c
User: _
Bot: d
User: i
Bot: a
User: l
Bot: o
User: g
Bot: u
User: e
Bot: s
Bot: I like acting, I hope to be an actor, what about you?



**Load tokenizer & model**

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token


**Tokenize dataset**

In [22]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

**Data collator**

In [23]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


**Use LoRA**

In [24]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


**Training configuration**

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./chatbot-lora",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    report_to="none"
)


**Trainer**

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)


**START FINE-TUNING**

In [27]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  "pin_memory_device is deprecated, the current accelerator will be used as the device,"
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


KeyboardInterrupt: 

**Saving chatbot**

In [28]:
model.save_pretrained("./chatbot-lora")
tokenizer.save_pretrained("./chatbot-lora")


('./chatbot-lora/tokenizer_config.json',
 './chatbot-lora/special_tokens_map.json',
 './chatbot-lora/vocab.json',
 './chatbot-lora/merges.txt',
 './chatbot-lora/added_tokens.json',
 './chatbot-lora/tokenizer.json')